# Which dual-stack peers that have been consistently peering with RIS collectors since the start of dual-stack operation of the respective Root Server?

Assumption:
- ~~starting time is selected on March 1st 2008, because on February 2008 6 root servers started to use IPv6~~ It is true that the majority of Root Servers started using IPv6 from 2008/03. However, RIS data shows that until 2010/10 there are many sort of empty routing information to some root servers. Hence, use 2010/10 as the starting date
- Currently, I only use 7 Root Servers (C, D, F, I, K, L, M)


### Attention
let's just ditch this idea. I mean, it is difficult to find common peers that present since the start of dual-stack operation for a particular root server (I only got few results), let alone to find common peers for *all* Root Servers

In [1]:
%matplotlib inline
from ggplot import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import requests
import pytz
import time
from datetime import datetime

matplotlib.style.use('ggplot')

In [5]:
start = datetime(2010, 10, 1, 1, 0, 0)
stop = datetime(datetime.today().year, datetime.today().month, 1, 1, 0, 0)

# root_list = ['c', 'd', 'f', 'i', 'k', 'l', 'm']
root_list = ['m']
# Root IP addresses used between 2008-03-01 to today
def root_prefix(root, timestamp):
    if root == 'a':
        return '198.41.0.4'
    if root == 'b':  # not anycasted. ignore
        return ''
    if root == 'c':
        return '192.33.4.12'
    if root == 'd':
        if timestamp < 1357171200:  # 2013-01-03
            return '128.8.10.90'
        else:
            return '199.7.91.13'
    if root == 'e':
        return '192.203.230.10'
    if root == 'f':
        return '192.5.5.241'
    if root == 'g':
        return '192.112.36.4'
    if root == 'h':
        if timestamp < 1448928000:  # 2015-12-01
            return '128.63.2.53'
        else:
            return '198.97.190.53'
    if root == 'i':
        return '192.36.148.17'
    if root == 'j':
        return '192.58.128.30'
    if root == 'k':
        return '193.0.14.129'
    if root == 'l':
        if timestamp < 1193875200:  # 2007-11-01
            return '198.32.64.12'
        else:
            return '199.7.83.42'
    if root == 'm':
        return '202.12.27.33'


def root_prefix6(root, timestamp):
    if root == 'a':
        if timestamp < 1201651200:
            return ''
        return '2001:503:BA3E::2:30'
    if root == 'b':  # not anycasted
        return ''
    if root == 'c':
        return '2001:500:2::C'
    if root == 'd':
        return '2001:500:2D::D'
    if root == 'e':
        return ''
    if root == 'f':
        if timestamp < 1201651200:
            return ''
        return '2001:500:2f::f'
    if root == 'g':
        return ''
    if root == 'h':
        if timestamp < 1201651200:
            return ''
        return '2001:500:1::53'
    if root == 'i':
        return '2001:7fe::53'
    if root == 'j':
        if timestamp < 1201651200:
            return ''
        return '2001:503:c27::2:30'
    if root == 'k':
        if timestamp < 1201651200:
            return ''
        return '2001:7fd::1'
    if root == 'l':
        if timestamp < 1458691200:  # 2016-3-23
            return '2001:500:3::42'
        else:
            return '2001:500:9f::42'
    if root == 'm':
        if timestamp < 1201651200:
            return ''
        return '2001:dc3::35'
    
############################
# Helper methods
############################
def deduplicate(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

def get_peers(prefix, timestamp):
    url = 'https://stat.ripe.net/data/bgp-state/data.json?resource={0}&timestamp={1}'.format(prefix, timestamp)
#     print('get_peers {}'.format(url))
    data = requests.get(url).json()
    data = data['data']['bgp_state']

    bgp_data = {}
    peers = []
    if data:
        for item in data:
            bgp_data[item['path'][0]] = list(deduplicate(item['path']))
            peers.append(item['path'][0])
#     else:
#         print('empty bgp_data: {}'.format(url))
#         pass
    return bgp_data, peers


In [8]:
utc = pytz.utc
res = []
for root in root_list:
    print('Working on {}-Root...'.format(root))
    temp = []
    cur_date = start
    
    while cur_date <= stop:
    #     print(cur_date)
        utc_dt = utc.localize(cur_date)
        timestamp = int(time.mktime(utc_dt.timetuple()))
        ip4 = root_prefix(root, timestamp)
        ip6 = root_prefix6(root, timestamp)
        data4, peer4 = get_peers(ip4, timestamp)
        data6, peer6 = get_peers(ip6, timestamp)
        
        if peer4 and peer6: # if peer4 and peer6 is not empty, then find the mutual peers and append to temp
            mutual_peers = set(peer4) & set(peer6)
            print('-- mutual_peers {}'.format(mutual_peers))
            if len(mutual_peers) > 12:
                temp.append(mutual_peers)
            else:
                print('-- root {} at {} has less than 12 mutual peers'.format(root, timestamp))

        year = cur_date.year + 1 if cur_date.month == 12 else cur_date.year
        month = 1 if cur_date.month == 12 else cur_date.month + 1
        cur_date = datetime(year, month, 1, 1, 0, 0)
    
    common_mutual_peers = set.intersection(*temp)
    print('- common mutual peers: {}'.format(common_mutual_peers))
    res.append(common_mutual_peers)

result = set.intersection(*res)
print(result)


Working on m-Root...
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=202.12.27.33&timestamp=1204329600
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=2001:dc3::35&timestamp=1204329600
-- mutual_peers {1280, 29449, 1930, 6667, 7575, 286, 2857, 24875, 559, 34225, 13237, 3257, 12859, 1853, 2497, 29636, 2914, 6762, 12779, 29686, 1916, 8447}
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=202.12.27.33&timestamp=1207008000
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=2001:dc3::35&timestamp=1207008000
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=202.12.27.33&timestamp=1209600000
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=2001:dc3::35&timestamp=1209600000
-- mutual_peers {34225}
-- root m at 1209600000 has less than 12 mutual peers
get_peers https://stat.ripe.net/data/bgp-state/data.json?resource=202.12.27.33&timestamp=1212278400
get_peers https://stat.ripe.net/data/bgp-state